<a href="https://colab.research.google.com/github/JoaquinGonzalezSimon/Data_science_and_ML_from_Medium/blob/main/230411_Number_of_Orders_Prediction_with_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This project was taken from the following link

https://thecleverprogrammer.com/2021/09/27/number-of-orders-prediction-with-machine-learning/

In [1]:
pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np

import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import lightgbm

In [6]:
data = pd.read_csv('https://raw.githubusercontent.com/amankharwal/Website-data/master/supplement.csv')
data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Date           188340 non-null  object 
 6   Holiday        188340 non-null  int64  
 7   Discount       188340 non-null  object 
 8   #Order         188340 non-null  int64  
 9   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 14.4+ MB


In [8]:
data.isnull().sum()

ID               0
Store_id         0
Store_Type       0
Location_Type    0
Region_Code      0
Date             0
Holiday          0
Discount         0
#Order           0
Sales            0
dtype: int64

In [9]:
data.describe()

,Store_id,Holiday,#Order,Sales
count,188340.000000,188340.000000,188340.000000,188340.000000
mean,183.000000,0.131783,68.205692,42784.327982
std,105.366308,0.338256,30.467415,18456.708302
min,1.000000,0.000000,0.000000,0.000000
25%,92.000000,0.000000,48.000000,30426.000000
50%,183.000000,0.000000,63.000000,39678.000000
75%,274.000000,0.000000,82.000000,51909.000000
max,365.000000,1.000000,371.000000,247215.000000


In [10]:
def pie_chart(series):
  pie = data[series].value_counts()
  x = pie.index
  y = pie.values

  fig = px.pie(data, values=y, names=x)
  fig.show()

In [12]:
data.columns

Index(['ID', 'Store_id', 'Store_Type', 'Location_Type', 'Region_Code', 'Date',
       'Holiday', 'Discount', '#Order', 'Sales'],
      dtype='object')

In [41]:
graph_cols = ['Store_Type', 'Location_Type', 'Holiday', 'Discount']

In [14]:
for i in graph_cols:
  pie_chart(i)

In [15]:
le = LabelEncoder()

In [42]:
encode_cols = graph_cols.copy()
encode_cols.remove('Holiday')
encode_cols

['Store_Type', 'Location_Type', 'Discount']

In [43]:
for col in encode_cols:
  data['le_'+col] = le.fit_transform(data[col])

In [44]:
data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales,le_Discount,le_Store_Type,le_Location_Type
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84,1,0,2
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12,1,3,1
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20,1,2,1
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16,1,1,2
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52,1,1,2


In [46]:
x = np.array(data[['le_Store_Type', 'le_Location_Type', 'le_Discount', 'Holiday']])
y = np.array(data['#Order'])

In [47]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((150672, 4), (37668, 4), (150672,), (37668,))

In [48]:
model = lightgbm.LGBMRegressor()
model.fit(xtrain, ytrain)

LGBMRegressor()

In [49]:
ypred = model.predict(xtest)
ypred

array([ 64.26160934,  36.6515288 ,  47.75772856, ..., 103.77916463,
        66.68691233,  54.39553805])

In [53]:
df = pd.DataFrame(data={'Predicted Orders': ypred.flatten()})
df.head()

,Predicted Orders
0,64.261609
1,36.651529
2,47.757729
3,45.327324
4,66.686912
